In [ ]:
# Alif Aqila Hakim / 220010086
# Crops Analysis and Parameter Preparation for Predictive Model Project

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import svm





In [ ]:
df = pd.read_csv("crop_grains_data.csv")
print(df.head())
print(df.info())
print(df.describe())

In [ ]:
df.dtypes

In [ ]:
# transform data
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

print(df.info())
print(df.head())

#NULLvalues
print(df.isnull().sum())

#clean data
uniqueVal=df['commodity'].unique()
print(uniqueVal)
uniqueVal=df['ticker'].unique()
print(uniqueVal)


In [ ]:
# visualisasi data dasar untuk memeriksa semuanya sudah benar atau belum dan utk mendapatkan perspektif data yang luas

#color palette
uniqueVal=df['commodity'].unique()
print(uniqueVal)

uniqueVal=df['ticker'].unique()
print(uniqueVal)

palette={'Corn':'yellow','Oat':'orange','KC HRW Wheat':'cyan','Rough Rice':'grey','Soybean Oil':'blue','Soybean':'green'}

plt.figure(1)
f1=plt.scatter(df['date'],df['open'],alpha=0.5, c=df['commodity'].map(palette),s=1)

Kecendrungan masing-masing tanaman panen terkait dengan open price sangat mirip namun ada beberapa perbedaan kecil yang menarik untuk diperhatikan:

1. Pada tahun 2020 beras mencapai puncaknya, tidak seperti tahun-tahun lainnya
2. Pada tahun 2000 hingga tahun 2004, beras mengalami penurunan sedangkan beras lainnya stabil
3. Oat mengalami penurunan pada tahun 2023 dibandingkan yang lainnya
4. Minyak kedelai sangat stabil dalam rentang waktu tersebut
5. Jagung dan gandum memiliki perilaku yang sangat mirip tetapi puncaknya lebih penting pada gandum
6. Kedelai dengan perilaku yang sangat mirip dibandingkan dengan kedelai lainnya mengalami peningkatan seiring berjalannya waktu

In [ ]:
def graph(type):
	corn=df[df['commodity']==type]
	plt.plot(corn['date'],corn['open'],c='b',lw=0.5)
	plt.plot(corn['date'],corn['high'],c='g',lw=0.5)
	plt.plot(corn['date'],corn['low'],c='g',lw=0.5)
	plt.plot(corn['date'],corn['close'],c='r',lw=0.5)

plt.figure(5)

graph('KC HRW Wheat')
graph('Rough Rice')
graph('Corn')
graph('Oat')
graph('Soybean Oil')
graph('Soybean')

bukan perubahan penting antara open, close, low, dan high

In [ ]:
#correlation matrix

correlation_matrix = df[['open', 'high','low','close']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')

korelasi yang sangat tinggi, dilanjut dengan mengecek value

In [ ]:
correlations = []
p_values = []
correlation, p_value = pearsonr(df['open'], df['high'])
correlations.append(correlation)
p_values.append(p_value)
correlation, p_value = pearsonr(df['open'], df['close'])
correlations.append(correlation)
p_values.append(p_value)
correlation, p_value = pearsonr(df['open'], df['low'])
correlations.append(correlation)
p_values.append(p_value)
correlation, p_value = pearsonr(df['high'],df['low'])
correlations.append(correlation)
p_values.append(p_value)
correlation, p_value = pearsonr(df['high'],df['close'])
correlations.append(correlation)
p_values.append(p_value)
correlation, p_value = pearsonr(df['low'], df['close'])
correlations.append(correlation)
p_values.append(p_value)

for i in range(0,6):
	print(f'{correlations[i]}')
	print(f'{p_values[i]:.10f}')

R^2 yang sangat tinggi dan nilai p<0,05 parameter ini memberikan informasi yang sama, terkait multilinearitas, sehingga cukup buat prediction model hanya dengan salah satunya saja

In [ ]:
# untuk memeriksa regresi dua parameter ini
plt.figure(33)
plt.scatter(df['open'],df['close'],s=1)
regression = LinearRegression()
x=df['open']
x=x.values.reshape(-1, 1)
y=df['close']
regression.fit(x,y)
Y_pred = regression.predict(x)
plt.plot(x, Y_pred, label='Regresi Linear', color='red')
r2 = r2_score(y, Y_pred)
m = regression.coef_[0]
b = regression.intercept_
equation_linear = f'linear: y = {m:.2f}x + {b:.2f}'
r2_text = f'R^2 = {r2:.2f}'
plt.text(0.2, 0.9, equation_linear, fontsize=12, transform=plt.gca().transAxes)
plt.text(0.2, 0.85, r2_text, fontsize=12, transform=plt.gca().transAxes)

dapat dicari parameter lain, variasi harga high dan low

In [ ]:
variance= np.array(df['high']-df['low'])
print(variance)
df['variance']=variance

def vargraph(type,fig):
	corn=df[df['commodity']==type]
	plt.figure(fig)
	plt.plot(corn['date'],corn['variance']/max(corn['variance']),c='b',lw=0.5)

vargraph('Corn',6)
vargraph('Oat',7)
vargraph('KC HRW Wheat',8)
vargraph('Rough Rice',9)
vargraph('Soybean Oil',10)
vargraph('Soybean',11)

Dimungkinkan untuk mengecek jika variasi berbeda untuk setiap jenis tanaman panen, parameter ini dapat berguna untuk ditambahkan ke dataframe

In [ ]:
plt.figure(34)
plt.scatter(df['open'],df['variance'],s=1)
regression = LinearRegression()
x=df['open']
x=x.values.reshape(-1, 1)
y=df['variance']
regression.fit(x,y)
Y_pred = regression.predict(x)
plt.plot(x, Y_pred, label='Regresi Linear', color='red')
r2 = r2_score(y, Y_pred)
m = regression.coef_[0]
b = regression.intercept_
equation_linear = f'linear: y = {m:.2f}x + {b:.2f}'
r2_text = f'R^2 = {r2:.2f}'
plt.text(0.2, 0.9, equation_linear, fontsize=12, transform=plt.gca().transAxes)
plt.text(0.2, 0.85, r2_text, fontsize=12, transform=plt.gca().transAxes)

Variance dan price tidak menunjukkan korelasi penting, cari kembali informasi yang berbeda

In [ ]:
plt.figure(35)

correlation_matrix = df[['open', 'high','low','close','variance']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')


corn=df[df['commodity']=='Corn']
corn2=df[df['commodity']=='Oat']
corn3=df[df['commodity']=='KC HRW Wheat']
corn4=df[df['commodity']=='Rough Rice']
corn5=df[df['commodity']=='Soybean Oil']
corn6=df[df['commodity']=='Soybean']

merged_df = pd.merge(corn,corn2, on='date', how='inner',suffixes=('_corn1', '_corn2'))
merged_df = pd.merge(merged_df,corn3, on='date', how='inner',suffixes=('_merged', '_corn3'))
merged_df = pd.merge(merged_df,corn4, on='date', how='inner',suffixes=('_merged2', '_corn4'))
merged_df = pd.merge(merged_df,corn5, on='date', how='inner',suffixes=('_merged3', '_corn5'))
merged_df = pd.merge(merged_df,corn6, on='date', how='inner',suffixes=('_merged4', '_corn6'))

merged_df.columns=['ticker_corn','com_corn','date','op_corn','hi_corn','low_corn','clo_corn','vol_corn','var_corn',
'ticker_Oat','com_Oat','op_Oat','hi_Oat','low_Oat','clo_Oat','vol_Oat','var_Oat',
'ticker_KC HRW Wheat','com_KC HRW Wheat','op_KC HRW Wheat','hi_KC HRW Wheat','low_KC HRW Wheat','clo_KC HRW Wheat','vol_KC HRW Wheat','var_KC HRW Wheat',
'ticker_Rough Rice','com_Rough Rice','op_Rough Rice','hi_Rough Rice','low_Rough Rice','clo_Rough Rice','vol_Rough Rice','var_Rough Rice',
'ticker_Soybean Oil','com_Soybean Oil','op_Soybean Oil','hi_Soybean Oil','low_Soybean Oil','clo_Soybean Oil','vol_Soybean Oil','var_Soybean Oil',
'ticker_Soybean','com_Soybean','op_Soybean','hi_Soybean','low_Soybean','clo_Soybean','vol_Soybean','var_Soybean']

print(merged_df.info())

seperti yang sudah dicek sebelumnya, keempat parameter memberikan informasi yang sama selain variance nya

In [ ]:
plt.figure(36)

correlation_matrix = merged_df[['op_corn','hi_corn','low_corn','clo_corn','vol_corn','var_corn',
'op_Oat','hi_Oat','low_Oat','clo_Oat','vol_Oat','var_Oat',
'op_KC HRW Wheat','hi_KC HRW Wheat','low_KC HRW Wheat','clo_KC HRW Wheat','vol_KC HRW Wheat','var_KC HRW Wheat',
'op_Rough Rice','hi_Rough Rice','low_Rough Rice','clo_Rough Rice','vol_Rough Rice','var_Rough Rice',
'op_Soybean Oil','hi_Soybean Oil','low_Soybean Oil','clo_Soybean Oil','vol_Soybean Oil','var_Soybean Oil',
'op_Soybean','hi_Soybean','low_Soybean','clo_Soybean','vol_Soybean','var_Soybean']].corr()

# op = open, clo = close, hi = high, lo = low

sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm')

In [ ]:
final_df = merged_df.drop(columns=['ticker_corn','hi_corn','low_corn','clo_corn',
'ticker_Oat','hi_Oat','low_Oat','clo_Oat',
'ticker_Rough Rice','hi_Rough Rice','low_Rough Rice','clo_Rough Rice',
'ticker_KC HRW Wheat','hi_KC HRW Wheat','low_KC HRW Wheat','clo_KC HRW Wheat',
'ticker_Soybean Oil','hi_Soybean Oil','low_Soybean Oil','clo_Soybean Oil',
'ticker_Soybean','hi_Soybean','low_Soybean','clo_Soybean'])

print(final_df.info())

plt.figure(37)

correlation_matrix = final_df[['op_corn','vol_corn','var_corn',
'op_Oat','vol_Oat','var_Oat',
'op_KC HRW Wheat','vol_KC HRW Wheat','var_KC HRW Wheat',
'op_Rough Rice','vol_Rough Rice','var_Rough Rice',
'op_Soybean Oil','vol_Soybean Oil','var_Soybean Oil',
'op_Soybean','vol_Soybean','var_Soybean']].corr()
print(correlation_matrix)

sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm')

Parameter ini dapat menyediakan database yang baik untuk membuat model prediksi

In [ ]:
work_df=df.drop(columns=['ticker','high','low','close'])
work_df['date'] = pd.to_datetime(df['date'], errors='coerce')
work_df['month']= work_df['date'].dt.month
work_df['year']= work_df['date'].dt.year
work_df['day']= work_df['date'].dt.day
print(work_df.info())

plt.figure(38)
correlation_matrix = work_df[['open','volume','variance','day','month','year']].corr()

sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm')

In [ ]:
# stratified split

stratified_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
X = work_df[['commodity', 'month', 'year','day', 'open', 'variance']]
y = work_df['volume']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=pd.qcut(y, q=4))

stats_train = y_train.describe()
stats_test = y_test.describe()
print(stats_train)
print(stats_test)

In [ ]:
#linear model

X = pd.get_dummies(X, columns=['commodity'], drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=pd.qcut(y, q=4))

stats_train = y_train.describe()
stats_test = y_test.describe()
print(stats_train)
print(stats_test)

multi_modelRegression = LinearRegression()
multi_modelRegression.fit(X_train, y_train)

y_pred = multi_modelRegression.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Coeficiente de Determinación (R²): {r2}')

coeficient = multi_modelRegression.coef_
intercept = multi_modelRegression.intercept_

print('Coeficient:', coeficient)
print('Intercept:', intercept)

nilai R^2= 0.434 MSE= 1.550.000.000 sehingga bukan model yang tepat

In [ ]:
#random forest

param_grid = {
    'n_estimators': [500],
    'max_depth': [50],
    'min_samples_split': [5],
    'min_samples_leaf': [2],
    'bootstrap': [True]
}
rf_model = RandomForestRegressor(random_state=42)

random_search = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=param_grid,
    n_iter=1, # Jumlah kombinasi acak untuk dicoba
    cv=5,         # Jumlah pemisahan untuk validasi silang
    random_state=42,
    n_jobs=-1     # menggunakan semua CPU core untuk searching
)


random_search.fit(X_train, y_train)

best_params = random_search.best_params_
best_model = random_search.best_estimator_

best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

Dihasilkan R^2= 0,913 MSE= 2.401.106.675

In [ ]:
# gradient boost

gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

gb_model.fit(X_train, y_train)

y_pred = gb_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

model ini menghasilkan R^2= 0.640 and a MSE= 988.000.000 sehingga model terbaik dari uji model untuk prediksi hasil panen (crop) adalah Random Forest karena hasil R^2 paling tinggi dibandingkan model lainnya sehingga random forest merupakan model yang paling cocok